# **Модуль А.  Парсинг и предобработка данных**

***Перед началом работы нужно импортировать необходимые библиотеки.***

In [1]:
import sqlite3
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import aiosqlite
import aiohttp
import asyncio
import logging
from datetime import datetime
from time import perf_counter
import nest_asyncio

### *1.1 Загрузка данных*
Создать базу данных, содержащую структуру, согласно исходным данным. Загрузить данные из предоставленных ресурсов. Данные необходимо объединить в один набор. Загрузить содержимое каждого набора в базу данных, предусмотреть обработку исключений при загрузке. В наборах данных для последующего обучения должны быть включены все записи, присутствующие в исходных наборах. Базу данных необходимо поддерживать в актуальном состоянии на протяжении всей работы. Для решения задачи должны использоваться также дополнительные источники.

***Первым делом нужно настроить базовое логирование (шаблон вывода сообщения) и задать имя БД и размер пакета данных. Это нужно для того, чтобы:***

    1) Выводилась дата и время вывода сообщения и сделать отчёт о проделанной работе;

    2) Создали БД с заданным именем (по желанию можно поменять в любой момент);

    3) Загружались не все данные, а по частям для предотвращения перегрузки системы и сервера API.

In [2]:
# Настройка логирования (шаблон вывода сообщения)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

DATABASE_NAME = 'taxi_trips.db' # Имя БД
BATCH_SIZE = 10000 # Размер пакета данных

***С базовым логированием разобрались. Теперь нужно в БД создать таблицу, где будем туда загружать запарсенные данные. Для этого напишем функцию, которая создаёт таблицу с заданной структурой или наполняет в ней новые данные, если существует. При этом нужно учесть момент, что если в одном источнике (датасете) найдутся столбцы, в структуре которой не указаны, они автоматически будут добавлены в таблицу.***

In [ ]:
async def create_tables(conn, columns=None):
    """
    Функция создаёт таблицу с заданной структурой, если её в БД нету
    """
    async with conn.cursor() as cursor:
        await cursor.execute('''
            CREATE TABLE IF NOT EXISTS taxi_trips (
                vendorid INTEGER,
                lpep_pickup_datetime TEXT,
                lpep_dropoff_datetime TEXT,
                store_and_fwd_flag TEXT,
                ratecodeid INTEGER,
                pickup_longitude REAL,
                pickup_latitude REAL,
                dropoff_longitude REAL,
                dropoff_latitude REAL,
                passenger_count INTEGER,
                trip_distance REAL,
                fare_amount REAL,
                extra REAL,
                mta_tax REAL,
                tip_amount REAL,
                tolls_amount REAL,
                improvement_surcharge REAL,
                total_amount REAL,
                payment_type INTEGER,
                trip_type INTEGER
            )
        ''')

        if columns:
            """
            Сохраняет все колонки из таблицы в переменную, перебирая их, и делает проверку на недостающие
            """
            await cursor.execute("PRAGMA table_info(taxi_trips)")
            existing_columns = {row[1].lower() for row in await cursor.fetchall()}

            for col in columns:
                """
                Если функция обнаружит недостающие колонки, она их добавит в таблицу
                """
                if col.lower() not in existing_columns:
                    await cursor.execute(f"ALTER TABLE taxi_trips ADD COLUMN {col.lower()} TEXT")
        await conn.commit()
        logging.info("Таблица \"taxi_trips\" создана/изменена.")

***Теперь можно потихоньку начать разрабатывать парсинг. Сначала разработаем функцию, которая получает данные из API с учётом:***

    1) offset (смещение);

    2) limit (лимит/ограничение);

    3) Обработки ошибок.

In [19]:
async def fetch_data_from_api(session, url, offset=0, limit=1000, retries=5):
    """
    Получает данные из API с учётом offset и limit
    """
    params = {'$limit': limit, '$offset': offset} # Параметры

    """
    Обрабатывает попытки отправить запрос на получение данных
    """
    for attempt in range(retries):
        try:
            async with session.get(url, params=params) as response:
                if response.status == 500:
                    logging.warning(f"Ошибка сервера (500) для {url} со смещением {offset}. Повторная попытка...")
                    await asyncio.sleep(2 ** attempt) # Останавливает запрос на некоторое время
                    continue
                response.raise_for_status()
                return await response.json()
        except aiohttp.ClientResponseError as e:
            logging.error(f"Запрос не удался: {e}. URL: {url}")
            if attempt == retries - 1:
                raise
        except Exception as e:
            logging.error(f"Неожиданная ошибка: {e}. Повторная попытка...")
            await asyncio.sleep(2 ** attempt) # Останавливает запрос на некоторое время
    
    """
    Если функция истратила все попытки, возвращает ошибку
    """
    logging.error(f"Не удалось получить данные из {url} после {retries} попыток.")
    return None

***Далее следует написать функцию, которая вставляет данные в БД пачками (batch).***

In [20]:
async def insert_batch_into_db(conn, table_name, batch):
    """
    Функция вставляет данные в БД пачками (batch)
    """
    if not batch: # Если нету данных
        return
    columns = sorted(batch[0].keys()) # Сортирует колонки по важности
    placeholders = ', '.join('?' for _ in columns) # Готовит данные для вставки
    query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"

    async with conn.cursor() as cursor:
        await cursor.executemany(query, [tuple(row.get(col) for col in columns) for row in batch]) # Вставляет данные
        await conn.commit()

***Когда прописали несколько базовых функций для работы парсинга, уже можно приступить к главной части - сама функция для парсинга. Она будет парсить данные из API и вставлять в БД (в таблицу).***

In [ ]:
async def parse_and_insert_trips(conn, url):
    """
    Функция парсит данные из API и вставляет в БД (в таблицу)
    """
    offset = 0
    all_columns = set() # Создаёт множество для исключения повторяющихся колонок
    async with aiohttp.ClientSession() as session:
        while True:
            logging.info(f"Извлечение данных со смещением={offset}, лимит={BATCH_SIZE}")
            data = await fetch_data_from_api(session, url, offset=offset, limit=BATCH_SIZE)
            if not data: # Если нет данных
                logging.info(f"Больше нет данных для извлечения из {url}.")
                break

            for row in data:
                all_columns.update({col.lower() for col in row.keys()})

            await create_tables(conn, all_columns)
            batch = []
            for row in data:
                try:
                    """
                    Если функция обнаружит колонки "datetime", она преобразует в подходящий формат
                    """
                    if 'lpep_pickup_datetime' in row:
                        row['lpep_pickup_datetime'] = datetime.fromisoformat(row['lpep_pickup_datetime']).strftime('%Y-%m-%d %H:%M:%S')
                    if 'lpep_dropoff_datetime' in row:
                        row['lpep_dropoff_datetime'] = datetime.fromisoformat(row['lpep_dropoff_datetime']).strftime('%Y-%m-%d %H:%M:%S')
                    batch.append(row)
                except (ValueError, KeyError): # Если нашлась недопустимая строка
                    logging.warning(f"Пропуск недопустимой строки: {row}")
                    continue

            await insert_batch_into_db(conn, 'taxi_trips', batch)
            """
            Если количество последних данных меньше пакета данных, функция выводит остаток
            """
            if len(data) < BATCH_SIZE:
                logging.info(f"Извлечение последних записей={len(data)} из {url}.")
                break
            offset += BATCH_SIZE # Увеличиваем смещение, чтобы продолжить парсинг

***Напоследок осталась функция, отвечающая за обработку URL-адресов API и запуск парсинга. Перед написанием обязательно предусмотреть несколько моментов:***

    1) Можно передать как один источник (в нашем случае, это датасет), так и несколько таких;

    2) Нужно запустить процесс выполнения всех задач параллельно для быстродействия.

In [22]:
async def main(dataset_urls):
    """
    Функция обрабатывет источники и запускает парсинг
    """
    conn = await aiosqlite.connect(DATABASE_NAME)
    await create_tables(conn)

    tasks = [parse_and_insert_trips(conn, url) for url in dataset_urls]
    await asyncio.gather(*tasks) # Запускает параллельно

    await conn.close()

In [ ]:
# Источники
datasets = [
    "https://data.cityofnewyork.us/resource/2np7-5jsg.json",
    "https://data.cityofnewyork.us/resource/gi8d-wdg5.json",
    "https://data.cityofnewyork.us/resource/hvrh-b6nb.json"
]
start_time = perf_counter()

# Запускаем парсер, предварительно обрабатывая исключение
try:
    asyncio.run(main(datasets))
    logging.info(f"Обработка данных завершена за {perf_counter() - start_time:.2f} секунд.")
except RuntimeError:
    nest_asyncio.apply()
    asyncio.run(main(datasets))
    logging.info(f"Обработка данных завершена за {perf_counter() - start_time:.2f} секунд.")

ЕСЛИ ВДРУГ САЙТ НЕ ВОЗОБНОВИТ РАБОТУ!!!

In [1]:
import os
import csv
import sqlite3
from glob import glob

In [2]:
DATA_DIR = 'datasets'  # Папка с CSV-файлами
DB_NAME = 'taxi_trips.db'
TABLE_NAME = 'taxi_trips'
BATCH_SIZE = 10000  # Размер пакета для вставки

In [3]:
def create_table(cursor, columns=None):
    """Создаёт таблицу, если её нет, и добавляет новые колонки, если они появились."""
    cursor.execute('''
            CREATE TABLE IF NOT EXISTS taxi_trips (
                vendorid INTEGER,
                tpep_pickup_datetime TEXT,
                tpep_dropoff_datetime TEXT,
                passenger_count INTEGER,
                trip_distance REAL,
                pickup_longitude REAL,
                pickup_latitude REAL,
                ratecodeid INTEGER,
                store_and_fwd_flag TEXT,
                dropoff_longitude REAL,
                dropoff_latitude REAL,
                payment_type INTEGER,
                fare_amount REAL,
                extra REAL,
                mta_tax REAL,
                tip_amount REAL,
                tolls_amount REAL,
                improvement_surcharge REAL,
                total_amount REAL
            )
        ''')
    
    if columns:
        cursor.execute(f"PRAGMA table_info({TABLE_NAME})")
        existing_columns = {row[1].lower() for row in cursor.fetchall()}

        for col in columns:
            if col.lower() not in existing_columns:
                cursor.execute(f"ALTER TABLE {TABLE_NAME} ADD COLUMN '{col.lower()}' TEXT")

In [4]:
def format_datetime(value):
    """Преобразует дату и время в формат YYYY-MM-DD HH:MM:SS."""
    try:
        return datetime.fromisoformat(value).strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        return value

In [5]:
def insert_batch(cursor, columns, batch):
    """Вставляет пакет строк в таблицу с обработкой исключений."""
    if not batch:
        return
    
    placeholders = ", ".join("?" for _ in columns)
    query = f"INSERT INTO {TABLE_NAME} ({', '.join(f'"{col}"' for col in columns)}) VALUES ({placeholders})"
    
    try:
        cursor.executemany(query, batch)
    except sqlite3.DatabaseError as e:
        print(f"Ошибка при вставке данных: {e}")

In [6]:
def process_files():
    files = glob(os.path.join(DATA_DIR, '*.csv'))
    if not files:
        print("Нет CSV файлов в указанной папке.")
        return
    
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()
    
    all_columns = set()
    
    for file_path in files:
        print(f"Обработка файла: {file_path}")
        
        try:
            with open(file_path, newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                if not reader.fieldnames:
                    print(f"Файл {file_path} не содержит заголовков, пропускаем.")
                    continue
                
                normalized_columns = {col.lower() for col in reader.fieldnames}  # Приводим названия к нижнему регистру
                all_columns.update(normalized_columns)
                create_table(cur, all_columns)
                
                batch = []
                for row in reader:
                    normalized_row = {col.lower(): row.get(col, None) for col in reader.fieldnames}  # Приводим ключи к нижнему регистру
                    
                    # Преобразуем дату/время в нужный формат
                    if 'lpep_pickup_datetime' in normalized_row:
                        normalized_row['lpep_pickup_datetime'] = format_datetime(normalized_row['lpep_pickup_datetime'])
                    if 'lpep_dropoff_datetime' in normalized_row:
                        normalized_row['lpep_dropoff_datetime'] = format_datetime(normalized_row['lpep_dropoff_datetime'])
                    
                    data = tuple(normalized_row.get(col, None) for col in all_columns)
                    batch.append(data)
                    
                    if len(batch) >= BATCH_SIZE:
                        insert_batch(cur, all_columns, batch)
                        conn.commit()
                        batch = []
                
                if batch:
                    insert_batch(cur, all_columns, batch)
                    conn.commit()
        except (csv.Error, OSError) as e:
            print(f"Ошибка при обработке файла {file_path}: {e}")
    
    conn.close()
    print("Загрузка данных завершена.")

In [ ]:
process_files()

### *1.2 Предобработка данных и выделение значимых атрибутов*
Для представленного набора данных предполагается применить методы кластерного анализа для создания групп, схожих по характеристикам. Такие кластеры в будущем помогут выявить динамику зависимости различных параметров для оптимизации ресурсов предприятия пассажирских перевозок.
Необходимо определить, какие атрибуты имеют наибольшее влияние на определение таких групп объектов (кластеров), и оставить только их для последующего обучения. Также необходимо обосновать выбор дополнительных атрибутов и причину исключения каких-либо данных из исходного набора.

***По заданию нужно провести кластерный анализ. Однако перед этим нужно определить наиболее значимые атрибуты, а именно:***

    1) passenger_count (Количество пассажиров);

    2) trip_distance (Расстояние поездки (в милях));

    3) fare_amount (Стоимость проезда (в долларах));

    4) total_amount (Общая сумма оплаты (включая чаевые, налоги, сборы)).

***Когда разобрались, можно приступать к работе. Для начала нужно написать функцию, которая создаёт подключение к БД и загружает строки пачками (batch_size) для ускорения.***

In [8]:
def load_data_in_batches(db_path, table_name, batch_size=200000, max_rows=None):
    """
    Функция загружает необходимые данные для кластеризации пачками
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    selected_columns = ["passenger_count", "trip_distance", "fare_amount", "total_amount"] # Значимые атрибуты
    cursor.execute(f"SELECT {', '.join(selected_columns)} FROM {table_name} LIMIT 1")
    columns = [desc[0] for desc in cursor.description]
    
    offset = 0 # Смещение
    total_loaded = 0 # Количество загруженных строк

    while max_rows is None or total_loaded < max_rows:
        cursor.execute(f"SELECT {', '.join(selected_columns)} FROM {table_name} LIMIT {batch_size} OFFSET {offset}")
        batch = cursor.fetchall()
        if not batch: # Если нет данных
            break
        yield columns, np.array(batch, dtype=np.float32)
        """
        Увеличиваем значения, чтобы продолжить загрузку
        """
        offset += batch_size
        total_loaded += batch_size
    
    conn.close()

***Теперь надо написать функцию, которая будет проводить кластерный анализ:***

    1) Сначала масштабируем данные (оно гарантирует, что каждый признак вносит равный вклад в расчеты расстояния);

    2) После масштабирования данных применить модель кластеризации.

***Также нужно сделать пояснение по поводу выбора атрибутов, а также причины исключения некоторых данных.***

In [9]:
def select_relevant_attributes(db_path, table_name, num_clusters=3):
    """
    Функция выполняет кластеризацию и выводит результат
    """
    feature_scores = defaultdict(float)
    scaler = MinMaxScaler() # Модель масштабируемости данных
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=10000, random_state=42) # Алгоритм (модель) кластеризации
    
    all_data = []
    
    # Загрузка данных по частям
    for columns, data in load_data_in_batches(db_path, table_name):
        if data.shape[1] < 3:
            continue
        
        """
        Вычисляет медиану
        """
        median_values = np.nanmedian(data, axis=0)
        mask = np.isnan(data)
        data[mask] = np.take(median_values, np.where(mask)[1])
        
        data_scaled = scaler.partial_fit(data).transform(data) # Масштабирует данные
        
        kmeans.partial_fit(data_scaled)
        
        all_data.append(data_scaled)

        feature_variances = np.var(data_scaled, axis=0)
        for i, col in enumerate(columns):
            feature_scores[col] += feature_variances[i]
    
    all_data = np.vstack(all_data)
    kmeans.fit(all_data) # Применяет модель кластеризации
    
    sorted_features = sorted(feature_scores.items(), key=lambda x: x[1], reverse=True)
    
    """
    Выводит результат
    """
    print("=== Наиболее значимые атрибуты ===")
    for i, (feature, score) in enumerate(sorted_features[:10], 1):
        print(f"{i}. {feature}: Вклад в кластеризацию = {score:.4f}")
    
    """
    Обосновывает выбор атрибутов
    """
    print("\n\nОбоснование выбора атрибутов:")
    print("Выбранные атрибуты были определены на основе внутрикластерного разброса значений.\nЧем выше дисперсия признака, тем больше он влияет на процесс кластеризации.\nОсновные показатели (количество пассажиров, расстояние поездки, стоимость проезда) напрямую связаны с динамикой пассажиропотока и важны для оптимизации транспортных ресурсов.")

    """
    Обосновывает причину исключения некоторых данных
    """
    print("\n\nПричина исключения некоторых данных:")
    print("Некоторые атрибуты (например, географические координаты, временные метки) были исключены, так как они не несут ключевой информации для текущей задачи кластеризации.\nОни могут быть полезны в других анализах (например, прогнозировании загруженности станций).")

In [ ]:
db_path = "taxi_trips.db" # Путь к БД
table_name = "taxi_trips" # Имя таблицы

select_relevant_attributes(db_path, table_name)

### *1.3 Описание структуры набора данных*
Для каждого атрибута подготовьте описание, содержащее текстовое представление (расшифровка, перевод, назначение). Выполнить проверку нормальности распределения значений в атрибутах. Для определения отклонений от нормы провести графическую и статистические оценки. Обосновать выбор методов оценки. На основе полученных результатов сделать вывод о типе распределения и мере скошенности.

***Перед началом работы нужно учесть несколько фактов:***

    1) Для описания каждого атрибута возьмём значительные;

    2) Для описания о количестве пустых значений возьмём все атрибуты;

    3) Для определения отклонений от нормы и распределния используем Shapiro-Wilk тест;

    4) На основе полученных данных делаем вывод, а также пояснение, почему выбрали тот или иной метод оценки.

***Разобравшись с фактами, начинаем работу. Напишем функцию, которая загружает данные из БД и возвращает массив значений.***

In [11]:
def load_data(db_path, table_name, columns):
    """
    Функция загружает необходимые данные из таблицы и вовращает массив значений
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(f"SELECT {', '.join(columns)} FROM {table_name}")
    data = np.array(cursor.fetchall(), dtype=np.float32)
    conn.close()
    return data

***Идём дальше! Создаём функции, каждая из которых отвечает за:***

    1) Вывод описании атрибутов (значимых) набора данных;

    2) Вывод количества пустых значений у каждого атрибута.

In [12]:
def describe_attributes():
    """
    Функция выводит описание значимых атрибутов
    """
    descriptions = {
        "passenger_count": "Количество пассажиров в поездке.\nНазначение: влияет на загруженность транспорта и тариф.\n",
        "trip_distance": "Расстояние поездки (в милях).\nНазначение: основной фактор стоимости и времени поездки.\n",
        "fare_amount": "Стоимость проезда (в долларах).\nНазначение: формируется на основе расстояния и тарифа.\n",
        "total_amount": "Общая сумма оплаты (включая чаевые, налоги, сборы).\nНазначение: итоговый платеж пассажира."
    }
    print("=== Описание атрибутов ===")
    for attr, desc in descriptions.items():
        print(f"{attr}: {desc}")

In [13]:
def describe_attributes_nulls(db_path, table_name):
    """
    Функция выводит количество пустых значений у каждого атрибута
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [row[1] for row in cursor.fetchall()]

    query = f"SELECT {', '.join([f"SUM(CASE WHEN {col} IS NULL OR {col} = '' THEN 1 ELSE 0 END) AS {col}_missing" for col in columns])} FROM {table_name}"
    cursor.execute(query)
    missing_counts = cursor.fetchone()
    
    print("\n\n=== Количество пустых значений у каждого атрибута ===")
    for i, col in enumerate(columns):
        print(f"{col}: Пропущенные значения = {missing_counts[i]}")
    
    conn.close()

***Теперь остался последний этап работы - по заданию нужно выполнить проверку нормальности распределения значений. Для этого создаём функцию, отвечающая за графические и статистические оценки.***

In [16]:
def analyze_distribution(data, columns):
    """
    Функция проводит проверку нормальности распределения значений
    """
    print("\n\n=== Анализ распределения ===")
    for i, col in enumerate(columns):
        values = data[:, i]
        values = values[~np.isnan(values)]
        
        """
        Вычисляет значение
        """
        p = stats.shapiro(values[:5000]).pvalue
        is_normal = p > 0.05
        
        """
        Проводит визуализацию
        """
        plt.figure(figsize=(6, 4))
        sns.histplot(values, kde=True, bins=50)
        plt.title(f"Распределение {col}\nShapiro-wilk p-value: {p:.4f} ({'Нормальное' if is_normal else 'Ненормальное'})")
        plt.xlabel(col)
        plt.ylabel("Частота")
        plt.show()
        
        """
        Выводит результат
        """
        print(f"Атрибут: {col}")
        print(f"Среднее: {np.mean(values):.2f}, Медиана: {np.median(values):.2f}, Среднеквадратическое отклонение: {np.std(values):.2f}")
        print(f"p-value теста Shapiro-wilk: {p:.4f} → {'Распределение нормальное' if is_normal else 'Распределение не нормальное'}\n")
    
    """
    Выводит обоснование выбора метода оценки
    """
    print("\nОбоснование выбора метода оценки:")
    print("Тест Shapiro-wilk используется для проверки нормальности распределения, так как он эффективен для небольших и средних выборок.\nВизуальный анализ (гистограммы и KDE) помогает дополнительно оценить характер распределения данных и выявить возможные выбросы.")

In [ ]:
db_path = "taxi_trips.db" # Путь к БД
table_name = "taxi_trips" # Имя таблицы
columns = ["passenger_count", "trip_distance", "fare_amount", "total_amount"] # Значимые атрибуты

describe_attributes()
describe_attributes_nulls(db_path, table_name)
data = load_data(db_path, table_name, columns)
analyze_distribution(data, columns)

### *1.4 Определение численных характеристик наборов данных*
Определите характеристики, удовлетворяющие следующим требованиям:

    1)	Общая загруженность станции

    2)	Пропускная способность в процентах по отношению к средней пропускной способности

    3)	Количественные характеристики входного и выходного оборудования

    4)	Временные интервалы поездки

    5)	Продолжительность поездки


***Перед началом работы нужно написать функцию, которая отвечает за обработку SQL-запросов и вовзращает результат.***

In [18]:
def execute_query(conn, query, params=()):
    """
    Функция обрабатывает SQL-запросы и возвращает результат
    """
    cursor = conn.cursor()
    cursor.execute(query, params)
    return cursor.fetchall()

***Затем создаём функцию, которая определяет числовые характеристики по требованиям, которые описанны выше. Для "Общей загруженности станции", "Пропускной способности" и "Количественных характеристик входного и выходного оборудования" выводим 10 строк для наглядности.***

In [ ]:
def compute_numeric_characteristics(db_path):
    """
    Функция вычисляет числовые характеристики по требованиям
    """
    print("=== Числовые характеристики ===")
    
    conn = sqlite3.connect(db_path)
    
    """
    1. Общая загруженность станций
    """
    station_load = execute_query(conn, "SELECT pickup_longitude, pickup_latitude, COUNT(*) FROM taxi_trips GROUP BY pickup_longitude, pickup_latitude LIMIT 10")
    print("\nОбщая загруженность станций (ТОП-10):")
    for lon, lat, count in station_load:
        print(f"Координаты: ({lon}, {lat}) — {count} поездок")
    
    """
    2. Пропускная способность станций
    """
    avg_passengers = execute_query(conn, "SELECT AVG(passenger_count) FROM taxi_trips")[0][0]
    capacity_percentage = execute_query(conn, "SELECT pickup_longitude, pickup_latitude, (AVG(passenger_count) * 100.0 / ?) FROM taxi_trips GROUP BY pickup_longitude, pickup_latitude LIMIT 10", (avg_passengers,))
    print("\nПропускная способность станций (ТОП-10):")
    for lon, lat, percentage in capacity_percentage:
        print(f"Координаты: ({lon}, {lat}) — {percentage:.2f}% от средней")
    
    """
    3. Количественные характеристики входного и выходного оборудования
    """
    equipment_count = execute_query(conn, "SELECT pickup_longitude, pickup_latitude, COUNT(*) FROM taxi_trips GROUP BY pickup_longitude, pickup_latitude LIMIT 10")
    print("\nОценка входного и выходного оборудования (ТОП-10):")
    for lon, lat, count in equipment_count:
        print(f"Координаты: ({lon}, {lat}) — {count} поездок (приблизительно столько единиц оборудования)")
    
    """
    4. Временные интервалы поездки
    """
    trip_intervals = execute_query(conn, "SELECT strftime('%H', lpep_pickup_datetime) AS hour, COUNT(*) FROM taxi_trips GROUP BY hour")
    print("\nРаспределение поездок по часам суток:")
    for hour, count in trip_intervals:
        print(f"{hour}:00 — {count} поездок")
    
    """
    5. Продолжительность поездки
    """
    avg_trip_duration = execute_query(conn, "SELECT AVG((julianday(lpep_dropoff_datetime) - julianday(lpep_pickup_datetime)) * 1440) FROM taxi_trips")[0][0]
    print(f"\nСредняя продолжительность поездки: {avg_trip_duration:.2f} минут")
    
    conn.close()

In [ ]:
db_path = "taxi_trips.db" # Путь к БД

compute_numeric_characteristics(db_path)

### *1.5 Формирование итогового набора данных*
Задача кластеризации предполагает определение наиболее предпочтительных условий для поездки. Поэтому необходимо переформировать набор данных таким образом, чтобы в наборе отражались все возможные маршруты с указанием всех возможных характеристик, определённых выше, а также других характеристик, полученных из внешних источников

***На основе полученных результатов формируем итоговую таблицу "final_trip_data", в которую будем загружать все возможные маршруты с указанием всех возможных характеристик, определённых выше, а также других характеристик, полученных из внешних источников.***

In [ ]:
def create_final_table(db_path):
    """
    Функция создаёт итоговую таблицу на основе полученных результатов
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS final_trip_data (
            trip_id INTEGER PRIMARY KEY,
            pickup_longitude REAL,
            pickup_latitude REAL,
            dropoff_longitude REAL,
            dropoff_latitude REAL,
            passenger_count INTEGER,
            trip_distance REAL,
            fare_amount REAL,
            total_amount REAL,
            trip_duration REAL,
            trip_hour INTEGER
        )
    """)
    
    cursor.execute("""
        INSERT INTO final_trip_data (pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude,
                                    passenger_count, trip_distance, fare_amount, total_amount, trip_duration, trip_hour)
        SELECT pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude,
               passenger_count, trip_distance, fare_amount, total_amount,
               (julianday(lpep_dropoff_datetime) - julianday(lpep_pickup_datetime)) * 1440 AS trip_duration,
               strftime('%H', lpep_pickup_datetime) AS trip_hour
        FROM taxi_trips
    """)
    
    conn.commit()
    conn.close()
    print("=== Итоговый набор данных создан и заполнен! ===")

In [ ]:
db_path = "taxi_trips.db" # Путь к БД

create_final_table(db_path)

### *1.6 Подготовка отчёта*
Подготовьте отчет о проделанной работе по итогам модуля, в котором будут представлены результаты, выводы и обоснования выбора по каждому разделу задания. В отчете также опишите содержимое результирующих файлов

***В результате всей модули я сформировал отчёт в виде блоков записей и комментариев к коду для всех вышеперечисленных пунктов.***